In [105]:
import pandas as pd
import numpy as np
from math import sqrt
import tensorflow as tf
import keras
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
import pyswarm
import math
import math
from sklearn.model_selection import train_test_split # to split the dataset for training and testing
from sklearn.linear_model import SGDRegressor
from scipy import linalg
from sklearn import metrics
import random
import pyswarm

In [106]:
df = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/ratings.csv')
movies = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/movies.csv')

In [107]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [108]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [109]:
new_df = pd.concat([movies.drop('genres', 1), movies['genres'].str.get_dummies(sep="|")], 1)

In [110]:
df = pd.merge(df, new_df, on='movieId')

In [111]:
def f(user):
    profile_1 = df[df['userId']==user]
    profile_1.drop(['movieId','title','timestamp','userId'], axis=1, inplace=True)
    profile_1["sum"] = profile_1.sum(axis=1)
    profile1_new = profile_1.loc[:,"rating":"Western"].div(profile_1["sum"], axis=0)
    return profile1_new

In [112]:
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]

    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.rating + w2*row.Action + w3*row.Adventure + w4*row.Animation + w5*row.Children + w6*row.Comedy + w7*row.Crime + w8*row.Documentary + w9*row.Drama + w10*row.Fantasy + w12*row.Horror + w13*row.IMAX + w14*row.Musical + w15*row.Mystery + w16*row.Romance + w18*row.Thriller + w19*row.War + w20*row.Western)

    return ans

In [113]:
lb = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
ub = [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255]

In [114]:
# implementation of PSO
# checking and defining weight inside the function; a random number between 0.5 and 1
def pso(fitness_func, lbound, ubound, swarm_size=40, max_iter=10, dimensions=20, phip=1.494, phig=1.494):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]

    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/100)
        v_max.append((ubound[i]-lbound[i])/100)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                r1=random.uniform(0,1)
                r2=random.uniform(0,1)
                weight = random.uniform(0.5, 1)
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
                if v[i][j] > v_max[j]:
                    v[i][j] = v_max[j]
                
                if v[i][j] < v_min[j]:
                    v[i][j] = v_min[j]


                # Kumud's code
                #checking the limit of velocity, if goes beyond then sack it
#                 if v[i][j] > v_max:
#                     vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j])
                                 
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [115]:
# function to predict weights of a particular user using Particle Swarm Optimization
def predict_weights_PSO(df, user):
    user_df = df[df['userId']==user]
    # estimating weights using pso
    xopt, fopt = pso(fitness_func, lb, ub, swarm_size=40,max_iter=40)
    return xopt

In [116]:
testUser = 85

In [117]:
def predict_weights_all_pso(df):
    user_weight_dict = {}
    i=0
    for user in df.userId.unique():
        i+=1
        xopt = predict_weights_PSO(df, user)
        user_weight_dict[user] = xopt
        print(i)
    return user_weight_dict

user_weight_dict = predict_weights_all_pso(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [118]:
# functions for finding similarity between two users based on their preferences to individual criteria
# based on euclidean distance
# where weights is a list of individual weights of user1
def euclidean_similarity_weighted(df, user1, user2, weights):
    s = pd.merge(df[df['userId']==user1], df[df['userId']==user2], how="inner", on="movieId")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    for index, row in s.iterrows():
           
        tempvar = (w1*((row.rating_x - row.rating_y)**2) + \
                w2*((row.Action_x - row.Action_y)**2) + \
                w3*((row.Adventure_x - row.Adventure_y)**2) + \
                w4*((row.Animation_x - row.Animation_y)**2) + \
                w5*((row.Children_x - row.Children_y)**2) + \
                w6*((row.Comedy_x - row.Comedy_y)**2) + \
                w7*((row.Crime_x - row.Crime_y)**2) + \
                w8*((row.Documentary_x - row.Documentary_y)**2) + \
                w9*((row.Drama_x - row.Drama_y)**2) + \
                w10*((row.Fantasy_x - row.Fantasy_y)**2) + \
                w12*((row.Horror_x - row.Horror_y)**2) + \
                w13*((row.IMAX_x - row.IMAX_y)**2) + \
                w14*((row.Musical_x - row.Musical_y)**2) + \
                w15*((row.Mystery_x - row.Mystery_y)**2) + \
                w16*((row.Romance_x - row.Romance_y)**2) + \
                w18*((row.Thriller_x - row.Thriller_y)**2) + \
                w19*((row.War_x - row.War_y)**2) + \
                w20*((row.Western_x - row.Western_y)**2)    
                  ) 
        tempvar = max(0, tempvar)
        tempvar = math.sqrt(tempvar)
        sum1 += tempvar
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity

In [119]:
def averageRating(user):
    rating = df[df['userId']==user]['rating'].values
    movie = df[df['userId']==user]['movieId'].values
    average = np.average(rating)
    return rating, movie, average

In [120]:
def similarUsers(user):
    k = 25
    sim_scores = []
    for i in df.userId.unique():
        s = euclidean_similarity_weighted(df, user, i, user_weight_dict[user])
        #s = correlation(user,i)
        #s = euclidean_similarity(user,i)
        sim_scores.append(s)
    simusers = {'users':df.userId.unique(), 'similarity_score':sim_scores}
    simusers = pd.DataFrame(simusers)
    simusers = simusers.nlargest(k, 'similarity_score')
    return simusers
similar_user = similarUsers(testUser)

In [121]:
def predictRating(user,movie):
    n = 0
    predRating = 0
    k = 25
    #s = similarUsers(user)
    for i in similar_user['users']:
        rating = df[(df['userId']==i) & (df['movieId'] == movie)]['rating'].values
        sim = similar_user[similar_user['users']==i]['similarity_score'].values
        if(len(rating)!=0):
            predRating = predRating + (rating * sim)
    predRating = predRating/k 
    rating, movie, average = averageRating(user)
    predRating = average+predRating
    return predRating

In [122]:
def getTopMovies(user):
    top_movies = []
    testUser = 1
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in df[df['userId']==user]['movieId'].values:
                if j not in top_movies:
                    top_movies.append(j)
    return top_movies

In [123]:
def recommendation():
    mo = []
    p = []
    m = getTopMovies(testUser)
    rec= []
    for j in range(0,50):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        print(j)
        for i in predRating:
            if (np.isscalar(i)):
                p.append(i)
            else:
                p.append(i[0])
    d = {'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    rating,movie,average = averageRating(testUser)
    for m in df_u['movieId']:
        if m not in movie:
            rec.append(m)
    return rec,df_u
    #return df_u

In [124]:
rec,df_u = recommendation()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [125]:
print('recommendations for user', testUser, ':\n')
for i in range(0,10):
    m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
    print(m[0])

recommendations for user 85 :

Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Bottle Rocket (1996)
Rob Roy (1995)
Canadian Bacon (1995)
Desperado (1995)
Billy Madison (1995)
Dumb & Dumber (Dumb and Dumber) (1994)
Star Wars: Episode IV - A New Hope (1977)
Tommy Boy (1995)


In [126]:
df_c = pd.merge(df,df_u, on=['movieId'])

In [127]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(df_c['rating'], df_c['predicted rating']))
print(rmse)

1.4385606020251223
